# Customer Churn Prediction & Analysis Tool

### Importing the libraries

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

### Reading the Dataset

In [ ]:
df = pd.read_csv('churn.csv')

### Frist look at the dataset

In [ ]:
df.head()
#df.shape
#df.info()
#df.describe()
#df["Exited"].value_counts(normalize=True)

### Splitting the dataset into indipendet variables(X) and dipendent variable (y)

In [ ]:
X = dataset.iloc[:, :-1].values
y = dataset.iloc[:, -1].values